In [3]:
# import libraries 
import pandas as pd
import numpy as np
import math
from numpy.linalg import norm

from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from pickle import dump, load

pd.set_option('display.max_columns', None)

np.random_state = 42

## Load in `sd_pp` Dataset

In [5]:
# load in df with the cluster labels
sd_pp = pd.read_csv('data/sd_pp', index_col = 0)
sd_pp

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,-1.152181,-0.561263,-0.534033,-0.728248,2.546108,0.605540,1.504754,-0.909516,-0.562984,-0.490416,3.516369,-0.408425,0.513223,-0.764034,-0.238620,0.038201,-0.972836,-0.581030,-0.581030,0.372804,0.368466,0.464525,0.272019,0.278668,0.342562,0.598182,0.0,0.970300,0.0,-0.16778,-0.183109
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### Get the `listing_url`

In [6]:
# load in df with the listing urls
sd_listings_url = pd.read_csv('data/sd_modeling_with_urls', index_col= 0)
sd_listings_url = sd_listings_url[['listing_url']]
sd_listings_url = sd_listings_url.reset_index(drop = 'index')
sd_listings_url

,listing_url
0,https://www.airbnb.com/rooms/33159143
1,https://www.airbnb.com/rooms/17138468
2,https://www.airbnb.com/rooms/21898446
3,https://www.airbnb.com/rooms/25948680
4,https://www.airbnb.com/rooms/1756516
...,...
13034,https://www.airbnb.com/rooms/20473217
13035,https://www.airbnb.com/rooms/17975465
13036,https://www.airbnb.com/rooms/4227110
13037,https://www.airbnb.com/rooms/17975942


In [7]:
# save as csv
path = "/Users/aryakbodkhe/Desktop/airbnb_recommendations-main/notebooks/"

sd_listings_url.to_csv(path + 'url_listings')

## Create a new dataframe for comparison listing

**Select a listing**

In [8]:
# selects a random listing from sd_clustered
random_listing = sd_pp.sample(n = 1)
random_listing

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
5830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,-0.155281,-0.561263,0.302897,0.323957,-0.237575,3.972615,0.062632,0.666552,0.728416,-0.309335,1.607093,0.389161,-0.370764,-0.764034,-0.135992,0.038201,-0.972836,-0.547715,-0.547715,0.683043,0.368466,-0.88476,0.272019,0.278668,0.342562,-0.717556,0.0,-1.030609,0.0,-0.16778,-0.183109


### Convert to Arrays

In [9]:
# convert single listing to an array
random_listing_array = random_listing.values

# convert all listings to an array
sd_pp_array = sd_pp.values

In [10]:
print(random_listing_array.shape)
print(sd_pp_array.shape) 

(1, 240)
(13039, 240)


## Determine Cosine Similarity

In [11]:
# define two lists or arrays to compare
A = np.squeeze(np.asarray(sd_pp_array))
B = np.squeeze(np.asarray(random_listing_array))
print("A:\n", A)
print("B:\n", B)
 
# compute cosine similarity
cosine = np.dot(A,B)/(norm(A, axis=1)*norm(B))
print("Cosine Similarity:\n", cosine)

A:
 [[ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 ...
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]]
B:
 [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.     

### Add Cosine Similarity to `sd_clustered`

In [12]:
# load in df with the cluster labels
sd_clustered = pd.read_csv('data/sd_clustered', index_col = 0)
sd_clustered.reset_index(drop = 'index')

,id,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,33159143,1,32.91736,-117.07635,Scripps Ranch,92131,House,Private room,1,1.0,1.0,1,Real Bed,1100.0,1100.0,1000.0,0.0,1.0,0.0,60.0,90.0,within a few hours,100.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
1,17138468,0,32.84067,-117.27443,La Jolla,92037,Apartment,Entire home/apt,1,2.0,2.0,3,Real Bed,1400.0,1400.0,0.0,0.0,1.0,0.0,30.0,1125.0,no response,0.0,0.0,1.0,1.0,0.0,2.0,4.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
2,21898446,0,32.79797,-117.24250,Pacific Beach,92109,Townhouse,Private room,1,1.0,1.0,1,Real Bed,1250.0,1250.0,0.0,0.0,1.0,0.0,30.0,30.0,no response,0.0,0.0,1.0,1.0,0.0,0.0,0.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,flexible,0.0,0.0
3,25948680,0,32.77545,-117.05923,College Area,92120,Apartment,Entire home/apt,1,1.0,1.0,1,Real Bed,1150.0,1150.0,0.0,0.0,1.0,0.0,31.0,40.0,no response,0.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
4,1756516,1,32.84619,-117.27558,La Jolla,92037,Condominium,Private room,1,1.0,1.0,1,Real Bed,110.0,110.0,200.0,0.0,1.0,50.0,180.0,365.0,within a day,67.0,0.0,6.0,1.0,1.0,2.0,4.0,90.0,7.0,7.0,8.0,8.0,9.0,10.0,0.0,0.0,0.0,strict_14_with_grace_period,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13034,20473217,4,32.78399,-117.25391,Mission Bay,92109,Villa,Entire home/apt,20,7.0,8.0,11,Real Bed,3000.0,3000.0,3000.0,0.0,20.0,0.0,1.0,1125.0,within an hour,95.0,1.0,86.0,1.0,1.0,0.0,0.0,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0.0,0.0,0.0,luxury_moderate,0.0,0.0
13035,17975465,4,32.80892,-117.26588,La Jolla,92037,House,Entire home/apt,20,7.0,10.0,13,Real Bed,2823.0,2823.0,1000.0,874.0,20.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,6.0,12.0,83.0,8.0,8.0,9.0,9.0,10.0,8.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
13036,4227110,4,32.79395,-117.23492,Mission Bay,92109,House,Entire home/apt,20,2.0,4.0,10,Real Bed,208.0,208.0,1000.0,332.0,4.0,25.0,1.0,1125.0,within an hour,96.0,0.0,11.0,1.0,1.0,142.0,284.0,92.0,9.0,9.0,9.0,9.0,10.0,8.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
13037,17975942,4,32.83598,-117.27878,La Jolla,92037,House,Entire home/apt,21,5.5,7.0,13,Real Bed,1305.0,1305.0,1000.0,800.0,21.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,4.0,8.0,80.0,9.0,8.0,10.0,8.0,10.0,9.0,0.0,1.0,0.0,super_strict_60,0.0,0.0


#### Adding in `listing_url`

In [13]:
# merge on index
sd_clustered = sd_clustered.join(sd_listings_url)

In [14]:
# add similarity 
rec = sd_clustered.copy()
rec['similarity'] = pd.DataFrame(cosine).values

In [15]:
# reorder column names
rec = rec[['id','listing_url', 'similarity', 'cluster_label', 'latitude', 'longitude',
       'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'number_of_reviews', 'number_of_stays',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']]

In [16]:
rec

,id,listing_url,similarity,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,33159143,https://www.airbnb.com/rooms/33159143,0.265325,1,32.91736,-117.07635,Scripps Ranch,92131,House,Private room,1,1.0,1.0,1,Real Bed,1100.0,1100.0,1000.0,0.0,1.0,0.0,60.0,90.0,within a few hours,100.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
1,17138468,https://www.airbnb.com/rooms/17138468,0.374418,0,32.84067,-117.27443,La Jolla,92037,Apartment,Entire home/apt,1,2.0,2.0,3,Real Bed,1400.0,1400.0,0.0,0.0,1.0,0.0,30.0,1125.0,no response,0.0,0.0,1.0,1.0,0.0,2.0,4.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
2,21898446,https://www.airbnb.com/rooms/21898446,0.366338,0,32.79797,-117.24250,Pacific Beach,92109,Townhouse,Private room,1,1.0,1.0,1,Real Bed,1250.0,1250.0,0.0,0.0,1.0,0.0,30.0,30.0,no response,0.0,0.0,1.0,1.0,0.0,0.0,0.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,flexible,0.0,0.0
3,25948680,https://www.airbnb.com/rooms/25948680,0.445730,0,32.77545,-117.05923,College Area,92120,Apartment,Entire home/apt,1,1.0,1.0,1,Real Bed,1150.0,1150.0,0.0,0.0,1.0,0.0,31.0,40.0,no response,0.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
4,1756516,https://www.airbnb.com/rooms/1756516,0.227405,1,32.84619,-117.27558,La Jolla,92037,Condominium,Private room,1,1.0,1.0,1,Real Bed,110.0,110.0,200.0,0.0,1.0,50.0,180.0,365.0,within a day,67.0,0.0,6.0,1.0,1.0,2.0,4.0,90.0,7.0,7.0,8.0,8.0,9.0,10.0,0.0,0.0,0.0,strict_14_with_grace_period,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13046,20473217,NaN,0.139449,4,32.78399,-117.25391,Mission Bay,92109,Villa,Entire home/apt,20,7.0,8.0,11,Real Bed,3000.0,3000.0,3000.0,0.0,20.0,0.0,1.0,1125.0,within an hour,95.0,1.0,86.0,1.0,1.0,0.0,0.0,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0.0,0.0,0.0,luxury_moderate,0.0,0.0
13047,17975465,NaN,0.184940,4,32.80892,-117.26588,La Jolla,92037,House,Entire home/apt,20,7.0,10.0,13,Real Bed,2823.0,2823.0,1000.0,874.0,20.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,6.0,12.0,83.0,8.0,8.0,9.0,9.0,10.0,8.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
13048,4227110,NaN,0.007893,4,32.79395,-117.23492,Mission Bay,92109,House,Entire home/apt,20,2.0,4.0,10,Real Bed,208.0,208.0,1000.0,332.0,4.0,25.0,1.0,1125.0,within an hour,96.0,0.0,11.0,1.0,1.0,142.0,284.0,92.0,9.0,9.0,9.0,9.0,10.0,8.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
13049,17975942,NaN,0.153607,4,32.83598,-117.27878,La Jolla,92037,House,Entire home/apt,21,5.5,7.0,13,Real Bed,1305.0,1305.0,1000.0,800.0,21.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,4.0,8.0,80.0,9.0,8.0,10.0,8.0,10.0,9.0,0.0,1.0,0.0,super_strict_60,0.0,0.0


## Get Recommendations!

In [17]:
# sort by highest similarity
rec.sort_values(by = ['similarity'], ascending = False).head(6)

,id,listing_url,similarity,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
5837,24329405,https://www.airbnb.com/rooms/5014151,1.000000,0,32.88289,-117.21074,Sorrento Valley,92121,Cottage,Entire home/apt,4,1.0,2.0,3,Real Bed,142.0,4260.0,300.0,165.0,4.0,5.0,30.0,1125.0,within a day,67.0,0.0,19.0,1.0,0.0,2.0,4.0,100.0,10.0,9.0,10.0,10.0,10.0,9.0,0.0,0.0,0.0,flexible,0.0,0.0
5812,35681369,https://www.airbnb.com/rooms/36355101,0.875241,0,32.79277,-117.24488,Mission Bay,92109,Condominium,Entire home/apt,4,1.0,2.0,2,Real Bed,146.0,4526.0,300.0,195.0,3.0,10.0,31.0,1125.0,within a day,67.0,0.0,19.0,1.0,0.0,0.0,0.0,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0.0,0.0,0.0,flexible,0.0,0.0
5773,36281470,https://www.airbnb.com/rooms/726684,0.853924,1,32.70861,-117.15581,East Village,92101,Condominium,Entire home/apt,4,2.0,2.0,2,Real Bed,189.0,5859.0,500.0,195.0,4.0,10.0,31.0,1125.0,within a day,67.0,0.0,19.0,1.0,0.0,0.0,0.0,95.0,10.0,10.0,10.0,10.0,10.0,9.0,0.0,0.0,0.0,moderate,0.0,0.0
8355,30518384,https://www.airbnb.com/rooms/37713066,0.839407,0,32.70848,-117.15567,East Village,92101,Condominium,Entire home/apt,5,2.0,2.0,2,Real Bed,151.0,4530.0,500.0,185.0,5.0,10.0,30.0,1125.0,within a day,67.0,0.0,19.0,1.0,0.0,1.0,2.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
5806,31801133,https://www.airbnb.com/rooms/960642,0.838784,0,32.78187,-117.13334,Serra Mesa,92108,Condominium,Entire home/apt,4,2.0,2.0,2,Real Bed,152.0,4560.0,500.0,225.0,5.0,10.0,30.0,1125.0,within a day,67.0,0.0,19.0,1.0,0.0,0.0,0.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
9066,23963146,https://www.airbnb.com/rooms/11160203,0.831202,0,32.84563,-117.27831,La Jolla,92037,Cottage,Entire home/apt,6,2.0,3.0,6,Real Bed,195.0,5850.0,0.0,0.0,1.0,0.0,30.0,120.0,a few days or more,0.0,0.0,1.0,1.0,0.0,6.0,12.0,100.0,10.0,9.0,10.0,10.0,10.0,9.0,0.0,0.0,0.0,flexible,0.0,0.0


### Get Recommendations Function

In [18]:
# selects a random listing from sd_clustered
random_listing = sd_pp.sample(n = 1)
random_listing

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
2973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.819881,0.543637,-0.534033,-0.728248,-0.475844,-0.43868,-0.246394,-0.575199,-0.132517,0.233906,-0.174899,-0.472386,0.513223,1.308842,-0.118887,0.038201,1.027922,1.351228,1.351228,0.527923,0.368466,0.464525,0.272019,0.278668,0.342562,0.598182,0.0,-1.030609,0.0,-0.16778,-0.183109


In [19]:
def get_recommendations(df, listing):
    """
    Takes in preprocessed dataframe and selected listing as inputs and gives top 5 (including listing)
    recommendations based on cosine similarity. 
    """
    # reset the index
    df = df.reset_index(drop = 'index')
    
    # convert single listing to an array
    listing_array = listing.values

    # convert all listings to an array
    df_array = df.values
    
    # get arrays into a single dimension
    A = np.squeeze(np.asarray(df_array))
    B = np.squeeze(np.asarray(listing_array))
    
    # compute cosine similarity 
    cosine = np.dot(A,B)/(norm(A, axis = 1)*norm(B))
    
    # add similarity into recommendations df and reset the index
    rec = sd_clustered.copy().reset_index(drop = 'index')
    rec['similarity'] = pd.DataFrame(cosine).values
    
    # reorder column names
    rec = rec[['id','listing_url', 'similarity', 'cluster_label', 'latitude', 'longitude',
       'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'number_of_reviews', 'number_of_stays',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']]
    
    # sort by top 5 descending
    return rec.sort_values(by = ['similarity'], ascending = False).head(6)


In [20]:
get_recommendations(sd_pp, random_listing)

,id,listing_url,similarity,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
2973,8307506,https://www.airbnb.com/rooms/12625311,1.000000,2,32.92119,-117.14494,Mira Mesa,92126,House,Private room,2,2.0,1.0,1,Real Bed,60.0,120.0,150.0,35.0,2.0,20.0,2.0,7.0,within an hour,100.0,1.0,22.0,1.0,1.0,116.0,232.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,moderate,0.0,0.0
2984,14340457,https://www.airbnb.com/rooms/17390402,0.999988,2,32.91997,-117.14214,Mira Mesa,92126,House,Private room,2,2.0,1.0,1,Real Bed,60.0,120.0,150.0,35.0,2.0,20.0,2.0,7.0,within an hour,100.0,1.0,22.0,1.0,1.0,117.0,234.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,moderate,0.0,0.0
3203,13577098,https://www.airbnb.com/rooms/14029274,0.998705,2,32.91891,-117.14232,Mira Mesa,92126,House,Private room,2,2.0,1.0,1,Real Bed,50.0,100.0,150.0,35.0,2.0,20.0,2.0,7.0,within an hour,100.0,1.0,22.0,1.0,1.0,109.0,218.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,moderate,0.0,0.0
102,2234293,https://www.airbnb.com/rooms/2234293,0.879431,2,32.90321,-117.13889,Mira Mesa,92126,House,Private room,1,1.0,1.0,1,Real Bed,48.0,960.0,100.0,20.0,1.0,20.0,20.0,1125.0,within an hour,100.0,1.0,6.0,1.0,1.0,166.0,332.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,moderate,0.0,0.0
4276,20196101,https://www.airbnb.com/rooms/21011929,0.868420,1,32.93211,-117.15265,Mira Mesa,92126,House,Private room,2,1.0,1.0,1,Real Bed,50.0,50.0,0.0,30.0,1.0,15.0,1.0,365.0,within an hour,100.0,1.0,4.0,1.0,1.0,46.0,92.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,moderate,0.0,0.0
4177,25217643,https://www.airbnb.com/rooms/28187241,0.856041,2,32.76148,-117.14493,West University Heights,92116,House,Private room,2,1.0,1.0,1,Real Bed,55.0,55.0,100.0,15.0,2.0,20.0,1.0,7.0,within an hour,91.0,1.0,2.0,1.0,1.0,131.0,262.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,moderate,0.0,0.0


## Simplified Recommendation System

### Load in `sd_trans` dataset

In [21]:
# load in df with the cluster labels
sd_trans = pd.read_csv('data/sd_trans', index_col = 0)
sd_trans.head()

,neighbourhood_cleansed,zipcode,property_type,room_type,bed_type,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
0,Scripps Ranch,92131,House,Private room,Real Bed,within a few hours,strict_14_with_grace_period,1,1.0,1.0,1,1100.0,1100.0,1000.0,0.0,1.0,0.0,60.0,90.0,100.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,0.0,0.0
1,La Jolla,92037,Apartment,Entire home/apt,Real Bed,no response,strict_14_with_grace_period,1,2.0,2.0,3,1400.0,1400.0,0.0,0.0,1.0,0.0,30.0,1125.0,0.0,0.0,1.0,1.0,0.0,2.0,4.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,0.0,0.0
2,Pacific Beach,92109,Townhouse,Private room,Real Bed,no response,flexible,1,1.0,1.0,1,1250.0,1250.0,0.0,0.0,1.0,0.0,30.0,30.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,0.0,0.0
3,College Area,92120,Apartment,Entire home/apt,Real Bed,no response,flexible,1,1.0,1.0,1,1150.0,1150.0,0.0,0.0,1.0,0.0,31.0,40.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0
4,La Jolla,92037,Condominium,Private room,Real Bed,within a day,strict_14_with_grace_period,1,1.0,1.0,1,110.0,110.0,200.0,0.0,1.0,50.0,180.0,365.0,67.0,0.0,6.0,1.0,1.0,2.0,4.0,90.0,7.0,7.0,8.0,8.0,9.0,10.0,0.0,0.0,0.0,0.0,0.0


In [22]:
sd_trans.columns

Index(['neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'bed_type', 'host_response_time', 'cancellation_policy', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'nightly_price', 'price_per_stay',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'host_response_rate',
       'host_is_superhost', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'number_of_reviews',
       'number_of_stays', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'require_guest_profile_picture',
       'require_guest_phone_verification'],
      dtype='object')

In [23]:
features_to_keep = ['neighbourhood_cleansed', 'property_type', 'room_type',
                   'accommodates', 'bathrooms', 'beds', 'nightly_price', 'review_scores_rating']

sd_simplified = sd_trans[features_to_keep]
sd_simplified.head()

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
0,Scripps Ranch,House,Private room,1,1.0,1,1100.0,98.0
1,La Jolla,Apartment,Entire home/apt,1,2.0,3,1400.0,100.0
2,Pacific Beach,Townhouse,Private room,1,1.0,1,1250.0,97.0
3,College Area,Apartment,Entire home/apt,1,1.0,1,1150.0,98.0
4,La Jolla,Condominium,Private room,1,1.0,1,110.0,90.0


In [24]:
# save as csv
path = "/Users/aryakbodkhe/Desktop/airbnb_recommendations-main/notebooks/"

sd_simplified.to_csv(path + 'sd_simplified')

### Construct a new preprocessing pipeline

In [25]:
# define nominal and ordinal features in the categorical columns
nom_cols = sd_simplified.select_dtypes(['object']).columns
print(nom_cols)

ordinal_cols = sd_simplified.select_dtypes(['category']).columns
print(ordinal_cols)

Index(['neighbourhood_cleansed', 'property_type', 'room_type'], dtype='object')
Index([], dtype='object')


We only have numeric and nominal features to preprocess in this simplified dataframe. 

In [26]:
# define numeric transformation pipeline that scales the numbers
numeric_pipeline = Pipeline([('numnorm', StandardScaler())]) 


# define a nominal transformation pipeline that OHE the cats
nominal_pipeline = Pipeline([('onehotenc', OneHotEncoder(categories= "auto", 
                                                         sparse = False, 
                                                         handle_unknown = 'ignore'))]) 

In [27]:
# construct column transformer for the selected columns with pipelines
ct = ColumnTransformer(transformers = [("nominalpipe", nominal_pipeline, ['neighbourhood_cleansed', 
                                                                          'property_type', 'room_type']),
                                       ("numericpipe", numeric_pipeline, sd_simplified.select_dtypes(['int', 'float']).columns)])



In [28]:
# save the NEW column transformer
dump(ct, open('/Users/aryakbodkhe/Desktop/airbnb_recommendations-main/notebooks/simple_column_transformer.pkl', 'wb'))

### Transforming the simplified dataset

In [29]:
sd_simplified_pp = pd.DataFrame(ct.fit_transform(sd_simplified))

/opt/homebrew/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [30]:
sd_simplified_pp.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.152181,-0.561263,-0.728248,2.546108,0.372804
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.152181,0.543637,0.323957,3.417825,0.683043
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.152181,-0.561263,-0.728248,2.981967,0.217684
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.152181,-0.561263,-0.728248,2.691394,0.372804
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.152181,-0.561263,-0.728248,-0.330558,-0.868154


### Construct a custom listing dataframe
Using user input, provide recommendations for a similar listing. 

In [31]:
import pandas as pd

# List of columns you want in the input DataFrame
columns_for_input = ['neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'beds', 'nightly_price', 'review_scores_rating']

# Create an empty DataFrame with the specified columns
input_df = pd.DataFrame(columns=columns_for_input)

# Display the DataFrame
input_df.head()

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating


In [32]:
# create an empty df with news column for input of values for recommendations
input_df = pd.DataFrame(columns = sd_simplified.columns)
input_df.head()

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating


In [33]:
neighborhood_input = "La Jolla"
property_input = "House"
room_input = "Entire home/apt "
accommodates = int(3)
bathrooms = float(2)
beds = int(2)
price = float(250)
rating = float(90)




In [34]:


# Assign input values to input_df columns
input_df = pd.DataFrame({
    'neighbourhood_cleansed': [neighborhood_input],
    'property_type': [property_input],
    'room_type': [room_input],
    'accommodates': [accommodates],
    'bathrooms': [bathrooms],
    'beds': [beds],
    'nightly_price': [price],
    'review_scores_rating': [rating]
})

# Convert to type int or float
input_df = input_df.astype({
    'accommodates': 'int',
    'bathrooms': 'float',
    'beds': 'int',
    'nightly_price': 'float',
    'review_scores_rating': 'float'
})

input_df


,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
0,La Jolla,House,Entire home/apt,3,2.0,2,250.0,90.0


In [35]:
# input numerical value for each predictor, convert to int value
# neighborhood_input = input("Enter Neighborhood: ")
# property_input = input("Enter Property Type: ")
# room_input = input("Enter Room Type: ")
# accommodates = int(input("Enter Accommodation Number: "))
# bathrooms = int(input("Number of Bathrooms: "))
# beds = int(input("Number of Beds: "))
# price = int(input("Nightly Price ($): "))
# rating = int(input("Review Score Rating: "))

# assign input values to input_df columns
input_df.loc[0, 'neighbourhood_cleansed' ] = neighborhood_input
input_df.loc[0, 'property_type'] = property_input
input_df.loc[0, 'room_type' ] = room_input
input_df.loc[0, 'accommodates'] = accommodates
input_df.loc[0, 'bathrooms' ] = bathrooms
input_df.loc[0, 'beds' ] = beds
input_df.loc[0, 'nightly_price'] = price
input_df.loc[0, 'review_scores_rating' ] = rating

# convert to type float, int
input_df = input_df.astype({"accommodates":"int",
                            "bathrooms":"float",
                            "beds":"int",
                            "nightly_price":"float",
                            "review_scores_rating":"float"})

input_df

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
0,La Jolla,House,Entire home/apt,3,2.0,2,250.0,90.0


### Get Recommendations for Simplified Dataset

In [36]:
def get_simplified_recommendations(df, listing):
    """
    Takes in preprocessed dataframe and preprocessed custom listing as inputs and gives top 5
    recommendations based on cosine similarity. 
    """
    # reset the index
    df = df.reset_index(drop = 'index')
    
    # preprocess the listing input
    listing_pp = pd.DataFrame(ct.transform(listing))
    
    # convert single listing to an array
    listing_array = listing_pp.values
    # convert all listings to an array
    df_array = df.values
    
    # get arrays into a single dimension
    A = np.squeeze(np.asarray(df_array))
    B = np.squeeze(np.asarray(listing_array))
    
    # compute cosine similarity 
    cosine = np.dot(A,B)/(norm(A, axis = 1)*norm(B))
    
    # add similarity into recommendations df and reset the index
    rec = sd_simplified.copy().reset_index(drop = 'index')
    rec['similarity'] = pd.DataFrame(cosine).values
    
    # add in listings_urls
    # merge on index
    rec = rec.join(sd_listings_url)
    
    # reorder column names
    rec = rec[['listing_url', 'similarity', 'neighbourhood_cleansed', 'property_type', 
               'room_type', 'accommodates', 'bathrooms', 'beds', 'nightly_price', 'review_scores_rating']]
    
    # sort by top 5 descending
    return rec.sort_values(by = ['similarity'], ascending = False).head(5)


In [37]:
# get top 5 most similar listings to user input
get_simplified_recommendations(sd_simplified_pp, input_df)

,listing_url,similarity,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
7541,https://www.airbnb.com/rooms/17975341,0.844941,La Jolla,House,Entire home/apt,4,2.0,2,187.0,84.0
10207,https://www.airbnb.com/rooms/17975377,0.765500,La Jolla,House,Entire home/apt,6,2.0,2,303.0,84.0
9352,https://www.airbnb.com/rooms/22479052,0.761201,La Jolla,House,Entire home/apt,6,2.0,2,375.0,87.0
508,https://www.airbnb.com/rooms/36678948,0.745642,La Jolla,House,Private room,1,1.5,1,70.0,92.0
212,https://www.airbnb.com/rooms/26637739,0.743697,La Jolla,House,Private room,1,1.0,1,250.0,80.0
